In [33]:
import tensorflow as tf

In [2]:
from tensorflow.examples.tutorials.mnist import input_data

In [34]:
mnist = input_data.read_data_sets('/tmp/data/', one_hot=True)

In [4]:
# parameter
learning_rate = 0.01
batch_size = 1000
drop_out = 0.7
training_epoch = 20

In [5]:
X_train = mnist.train.images
y_train = mnist.train.labels

In [6]:
X_train.shape, y_train.shape

((55000, 784), (55000, 10))

In [7]:
X = tf.placeholder(tf.float32, shape=[None, 784])
Y = tf.placeholder(tf.float32, shape=[None, 10])
keep_prob = tf.placeholder(tf.float32)

In [8]:
X_input = tf.reshape(X, shape=[-1, 28, 28, 1])

In [9]:
W1 = tf.Variable(tf.random_normal([5, 5, 1, 32]), tf.float32)

In [10]:
L1 = tf.nn.conv2d(X_input, W1, strides=[1, 1, 1, 1], padding='SAME')

In [11]:
L1 = tf.nn.relu(L1)

In [12]:
L1 = tf.nn.max_pool(L1, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='VALID')

In [13]:
L1.shape

TensorShape([Dimension(None), Dimension(14), Dimension(14), Dimension(32)])

In [14]:
W2 = tf.Variable(tf.random_normal([5, 5, 32, 64]), tf.float32)

In [15]:
L2 = tf.nn.conv2d(L1, W2, strides=[1, 1, 1, 1], padding='SAME')

In [16]:
L2 = tf.nn.relu(L2)

In [17]:
L2 = tf.nn.max_pool(L2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='VALID')

In [18]:
L2.shape

TensorShape([Dimension(None), Dimension(7), Dimension(7), Dimension(64)])

In [19]:
L2 = tf.layers.flatten(L2)

In [20]:
W3 = tf.Variable(tf.random_normal([7*7*64, 1024]))
b3 = tf.Variable(tf.random_normal([1024]))

In [21]:
L3 = tf.matmul(L2, W3) + b3

In [22]:
L3 = tf.nn.dropout(L3, keep_prob)

In [23]:
L3.shape

TensorShape([Dimension(None), Dimension(1024)])

In [24]:
W4 = tf.Variable(tf.random_normal([1024, 10]))
b4 = tf.Variable(tf.random_normal([10]))

In [25]:
L4 = tf.matmul(L3, W4) + b4

In [26]:
hypothesis = tf.nn.softmax(L4)

In [27]:
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=L4, labels=Y))

In [28]:
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

In [29]:
prediction = tf.argmax(hypothesis, axis=1)
actual = tf.argmax(Y, axis=1)
accuracy = tf.reduce_mean(tf.cast(tf.equal(prediction, actual), tf.float32))

In [30]:
tf.summary.scalar('cost', cost)
tf.summary.scalar('accuracy', accuracy)
summaries = tf.summary.merge_all()

In [32]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    summary_writer = tf.summary.FileWriter('logs/05_Convolution_Neural_Network_With_Mnist_Dataset', sess.graph)
    num_batch = int(mnist.train.num_examples / batch_size)
    for epoch in range(training_epoch):
        avg_cost = 0
        for b in range(num_batch):
            batch_xs, batch_ys = mnist.train.next_batch(batch_size)
            cost_val, _, accuracy_val, summary_ = sess.run([cost, optimizer, accuracy, summaries], feed_dict={X: batch_xs, Y: batch_ys, keep_prob: drop_out})
            avg_cost += cost_val / num_batch
        print("epoch {0}, cost = {1:.5f}, accuracy = {2:.5f}".format(epoch, cost_val, accuracy_val))
        summary_writer.add_summary(summary_, global_step=epoch)
    accuracy_val = sess.run(accuracy, feed_dict={X: mnist.test.images, Y: mnist.test.labels, keep_prob: 1.0}) 
    print("result, accuracy = {:.5f}".format(accuracy_val))

epoch 0, cost = 1077.04602, accuracy = 0.91300
epoch 1, cost = 478.95016, accuracy = 0.94800
epoch 2, cost = 323.35843, accuracy = 0.95200
epoch 3, cost = 150.60173, accuracy = 0.96800
epoch 4, cost = 233.73235, accuracy = 0.95800
epoch 5, cost = 121.11667, accuracy = 0.97900
epoch 6, cost = 101.87227, accuracy = 0.97200
epoch 7, cost = 144.88109, accuracy = 0.97100
epoch 8, cost = 153.92566, accuracy = 0.96700
epoch 9, cost = 65.55613, accuracy = 0.98100
epoch 10, cost = 78.32659, accuracy = 0.97900
epoch 11, cost = 67.79881, accuracy = 0.98000
epoch 12, cost = 37.62396, accuracy = 0.98400
epoch 13, cost = 42.64349, accuracy = 0.98800
epoch 14, cost = 56.52322, accuracy = 0.98400
epoch 15, cost = 66.25916, accuracy = 0.97800
epoch 16, cost = 42.95956, accuracy = 0.98700
epoch 17, cost = 39.50154, accuracy = 0.98900
epoch 18, cost = 38.71816, accuracy = 0.98600
epoch 19, cost = 33.52143, accuracy = 0.98900
result, accuracy = 0.98140
